### Sudoku Implementation:
Class sudoku includs functions that help fill empty slots in a sudoku board.

(By Shengyi Wu)

In [1]:
import numpy as np
import random
from collections import Counter
import time

In [2]:
class sudoku:
  """A class of functions to fill Sudoku board"""
  def __init__(self):
    """Initiate the class with an array block which defines the block number that
    each element belongs to"""
    self.blocks = np.array([[1,1,1,2,2,2,3,3,3],
                            [1,1,1,2,2,2,3,3,3],
                            [1,1,1,2,2,2,3,3,3],
                            [4,4,4,5,5,5,6,6,6],
                            [4,4,4,5,5,5,6,6,6],
                            [4,4,4,5,5,5,6,6,6],
                            [7,7,7,8,8,8,9,9,9],
                            [7,7,7,8,8,8,9,9,9],
                            [7,7,7,8,8,8,9,9,9]])
  
  def separate_index(self, idx):
    """get x-index and y-index from np.where funciton
       idx: np.where() result (index 0 is the x-axis index, index 1 is the y-axis index)"""
    X = []
    Y = []
    for l in range(len(idx[0])):
      x_i = idx[0][l]
      X.append(x_i)
      y_i = idx[1][l]
      Y.append(y_i)
    return X,Y
  
  def index_block(self, y):
    """Get the block numbers of elements in the same column
       y: column index (range 0 to 8)"""
    if y >= 0 and y <= 2:
      return [1,4,7]
    elif y >=3 and y <= 5:
      return [2,5,8]
    else:
      return [3,6,9]

  def rank_col(self, current_board):
    """Sort the order of columns based on the number of non-empty elements"""
    no_zero_mat = {}
    correct_index = []
    for i in range(9):
      curr_col = current_board[:,i]
      no_zero_mat[i] = [i for i in curr_col if i != 0]
    for k in sorted(no_zero_mat, key=lambda k: len(no_zero_mat[k]), reverse=True):
      correct_index.append(k)
    return correct_index

  def possibility(self, current_board):
    """Initiate the possible numbers to fill in each empty slot to be 1 to 9"""
    all_zeros = {}
    for i in range(9):
      for j in range(9):
        if current_board[i][j] == 0:
          X = i
          Y = j
          pos = list(np.arange(1,10))
          all_zeros[tuple([X, Y])] = pos
    return all_zeros


  def valid_col(self, current_board, y):
    """Test if the column is valid (consists of number 1 to 9)
       y: column index (range 0 to 8)"""
    col = current_board[:,y]
    col_comp = set(list(col)) == set(list(np.arange(1,10)))
    return col_comp

  
  def valid_board(self, current_board):
    """Test if the current board is valid
       - Each column consists of number 1 to 9
       - Each row consists of number 1 to 9
       - Each block consists of number 1 to 9"""
    row_result = []
    col_result = []
    block_result = []
    for i in range(9):
      row_comp = set(list(current_board[i,:])) != set(list(np.arange(1,10)))
      row_result.append(row_comp)
      col_comp = set(list(current_board[:,i])) != set(list(np.arange(1,10)))
      col_result.append(col_comp)

      same_block = np.where(self.blocks == i+1)
      X, Y = self.separate_index(same_block)
      block = [current_board[i][j] for i,j in zip(X,Y)]
      block_comp = set(block) != set(list(np.arange(1,10)))
      block_result.append(block_comp)
    total_result = row_result + col_result + block_result
    if sum(total_result) == 0:
      return True
    else:
      return False


  def fill_with_confidence(self, current_board):
    """The fill_with_confidence function fills in empty slots that we are 100% definite
    with only one answer. First, initiate a list of number 1 to 9 as
    possible answers. Then, delete those numbers that have already existed in the 
    same columns, same rows, or same blocks. For each slots, if there's only one number
    left in the list of possible numbers, fill the slot with the number. If there are
    multiple numbers left in the list, save the slot index and the list of possible 
    candidates into a dictionary. The fill_will_confidence function keeps filling till
    there is no change on the board."""
    possibilities = self.possibility(current_board)
    curr_board = np.zeros(shape=(9,9))
    Cs = {}
    while current_board.any() != curr_board.any():
      curr_board = current_board
      c = {}
      for col in range(len(self.rank_col(current_board))):
        i = self.rank_col(current_board)[col]
        curr_column = np.zeros(9)
        while curr_column.any() != current_board[:,i].any():
          curr_column = current_board[:,i]
          candidates = {}
          for j in range(1,10):
            curr_col = current_board[:,i]
            curr_check = j
            if curr_check in curr_col:
              continue
            else:
              block_idx = self.index_block(i)
              for b in block_idx:
                same_blocks = np.where(self.blocks == b)
                X, Y = self.separate_index(same_blocks)
                Index = [tuple([i,j]) for i,j in zip(X,Y)]
                block = np.array([current_board[i][j] for i,j in zip(X,Y)]).reshape((3,3))
                if curr_check in block:
                  for key in possibilities.keys():
                    if key in Index and curr_check in possibilities[key]:
                      curr_value = possibilities[key]
                      possibilities[key] = list(set(curr_value) - set([curr_check]))
              rest_col_index = list(set(list(np.arange(0,9))) - set([i]))
              rest_col = current_board[:,rest_col_index]
              for k in range(9):
                for l in rest_col_index:
                  if current_board[k][l] == curr_check:
                    if (k,i) in possibilities.keys() and curr_check in possibilities[(k,i)]:
                      curr_value = possibilities[(k,i)]
                      possibilities[(k,i)] = list(set(curr_value) - set([curr_check]))

              POS = []
              for ks in possibilities.keys():
                if ks[1] == i:
                  if curr_check in possibilities[ks]:
                    POS.append(ks)
              if len(POS) == 1:
                x_idx = POS[0][0]
                y_idx = POS[0][1]
                current_board[x_idx][y_idx] = curr_check
              elif len(POS) > 1:
                for p in range(len(POS)):
                  x_idx = POS[p][0]
                  y_idx = POS[p][1]
                  if (x_idx,y_idx) not in candidates.keys():
                      candidates[(x_idx, y_idx)] = [curr_check]
                  else:
                    candidates[(x_idx, y_idx)].append(curr_check)
                
        c.update(candidates)
        Cs = c

    sure_keys = []
    already_filled = []

    for key, item in Cs.items():
      if current_board[key[0]][key[1]] != 0:
        already_filled.append(key)
    for k in already_filled:
      if k in Cs.keys():
        Cs.pop(k)

    for key, item in Cs.items():
      if current_board[key[0]][key[1]] == 0:
        if len(item) == 1:
          current_board[key[0]][key[1]] = item[0]
          sure_keys.append(key)
    for k1 in sure_keys:
      if k1 in Cs.keys():
        Cs.pop(k1)
    return current_board, Cs



  def fill_random(self, current_board, Cs):
    """The fill_random function fill the rest empty slots with randomly chosen
    number from the intersection of the list of candidate numbers and updated 
    possible numbers based on numbers existing in the same row, column, and block."""
    while self.valid_board(current_board) == False:
      col = 0
      num_col = 9
      col_len = 9
      while col < num_col:
        i = self.rank_col(current_board)[col]
        ele = 0
        n = 0
        while ele < col_len and self.valid_col(current_board, i) == False and n < 15:
          if current_board[:,i][ele] == 0:
            block_num = self.blocks[ele][i]
            same_block = np.where(self.blocks == block_num)
            X, Y = self.separate_index(same_block)
            block = [current_board[i][j] for i,j in zip(X,Y)]
            left_ele = list(set(list(np.arange(1,10))) - set(list(current_board[:,i])) - set(list(current_board[ele,:])) - set(block))
            possible = list(set(Cs[(ele, i)]) & set(left_ele))
            if len(possible) != 0:
              current_board[ele][i] = random.choice(possible)
              ele += 1
              n = n
            else:
              for item in Cs.keys():
                if item[1] == i:
                  x = item[0]
                  current_board[x][i] = 0
              ele = 0
              n += 1
          else:
            ele += 1
        if n == 15:
          cols = self.rank_col(current_board)[0:col]
          for ite in Cs.keys():
            if ite[1] in cols:
              current_board[ite[0]][ite[1]] = 0
          col = 0
        else:
          col += 1
    return current_board
  
  def fill(self, current_board):
    current_board, Cs = self.fill_with_confidence(current_board)
    if self.valid_board(current_board) == True:
      return current_board
    else:
      current_board = self.fill_random(current_board, Cs)
      return current_board

### Examples:

In [13]:
mat1 = np.array([[1,3,0,0,9,0,0,5,0],
[0,4,0,5,0,0,3,0,0],
[0,0,9,0,0,0,0,0,1],
[9,0,0,0,0,7,4,0,0],
[0,6,0,0,8,0,0,9,0],
[0,0,1,4,0,0,0,0,8],
[3,0,0,0,0,0,1,0,0],
[0,0,7,0,0,2,0,3,0],
[0,9,0,0,1,0,0,4,6]])

example = sudoku()

## count time
start = time.time()
example.fill(mat1)
end = time.time()
print(end - start)

1.1569509506225586


In [14]:
example.fill(mat1)

array([[1, 3, 2, 6, 9, 8, 7, 5, 4],
       [8, 4, 6, 5, 7, 1, 3, 2, 9],
       [5, 7, 9, 3, 2, 4, 6, 8, 1],
       [9, 5, 8, 2, 6, 7, 4, 1, 3],
       [4, 6, 3, 1, 8, 5, 2, 9, 7],
       [7, 2, 1, 4, 3, 9, 5, 6, 8],
       [3, 8, 4, 9, 5, 6, 1, 7, 2],
       [6, 1, 7, 8, 4, 2, 9, 3, 5],
       [2, 9, 5, 7, 1, 3, 8, 4, 6]])

In [5]:
mat2 = np.array([[0,0,0,2,6,0,9,5,0],
[5,0,0,0,0,8,2,0,0],
[0,1,2,0,0,0,0,0,7],
[0,4,0,0,5,6,0,0,0],
[0,5,0,0,0,0,0,6,0],
[0,0,0,4,1,0,0,2,0],
[7,0,0,0,0,0,8,3,0],
[0,0,4,3,0,0,0,0,2],
[0,8,1,0,4,2,0,0,0]])

## count time
start = time.time()
example.fill(mat2)
end = time.time()
print(end - start)

0.5932154655456543


In [6]:
example.fill(mat2)

array([[8, 3, 7, 2, 6, 4, 9, 5, 1],
       [5, 9, 6, 1, 7, 8, 2, 4, 3],
       [4, 1, 2, 9, 3, 5, 6, 8, 7],
       [2, 4, 3, 8, 5, 6, 1, 7, 9],
       [1, 5, 9, 7, 2, 3, 4, 6, 8],
       [6, 7, 8, 4, 1, 9, 3, 2, 5],
       [7, 2, 5, 6, 9, 1, 8, 3, 4],
       [9, 6, 4, 3, 8, 7, 5, 1, 2],
       [3, 8, 1, 5, 4, 2, 7, 9, 6]])

In [7]:
mat3 = np.array([[2,0,9,0,0,0,4,8,0],
[8,0,3,0,9,6,0,0,0],
[5,0,0,8,4,0,9,2,0],
[7,0,0,3,0,4,0,0,0],
[1,0,0,0,5,0,0,0,4],
[0,0,0,7,0,2,0,0,8],
[0,7,6,0,2,3,0,0,5],
[0,0,0,4,6,0,7,0,2],
[0,2,5,0,0,0,6,0,9]])

## count time
start = time.time()
example.fill(mat3)
end = time.time()
print(end - start)

0.02583479881286621


In [8]:
example.fill(mat3)

array([[2, 1, 9, 5, 3, 7, 4, 8, 6],
       [8, 4, 3, 2, 9, 6, 1, 5, 7],
       [5, 6, 7, 8, 4, 1, 9, 2, 3],
       [7, 9, 2, 3, 8, 4, 5, 6, 1],
       [1, 3, 8, 6, 5, 9, 2, 7, 4],
       [6, 5, 4, 7, 1, 2, 3, 9, 8],
       [4, 7, 6, 9, 2, 3, 8, 1, 5],
       [9, 8, 1, 4, 6, 5, 7, 3, 2],
       [3, 2, 5, 1, 7, 8, 6, 4, 9]])